# Get Songs from Spotify Playlist

In [ ]:
# Libraries
import requests
import re
import json
import config
import matplotlib.pyplot as plt
import random

In [ ]:
# Get token
url = "https://accounts.spotify.com/api/token"

payload=f'client_id={config.CLIENT_ID}&client_secret={config.CLIENT_SECRET}&grant_type=client_credentials'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': '__Host-device_id=AQA1E_8u1HHZzuE7O9d_fV4YwanfNozg6q_mdfweHpowhpZGWG9saI45tX6W5no4APxdKSdgb0ZqexZNfHeYiXVZhLhlP5JYj0M; sp_tr=false'
}

response = requests.request("POST", url, headers=headers, data=payload)

AccessToken = json.loads(response.text)['access_token']
AccessToken

## Access Token
Access token is required to call Spotify API. In this case, the Access token needs to be refreshed every time the request is called. The Spotify credential used is the creator of the code.

In [ ]:
# Link to Playlist
link = 'https://open.spotify.com/playlist/6K0CRmpOM7YtMbCvmKM2mN?si=308fd53745834a27'

In [ ]:
# Get ID from Link to Playlist
playlist_id = re.search(r'(?<=playlist/)\w+', link).group()

playlist_id

In [ ]:
url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

payload={}
headers = {
  'Authorization': f'Bearer {AccessToken}'
}

playlistDetails = requests.get(url, headers=headers, data=payload).json()

print(f"Playlist Name: {playlistDetails['name']}")

In [ ]:
# Get Tracks inside the playlist
import requests

url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

payload={}
headers = {
  'Authorization': f'Bearer {AccessToken}'
}

response = requests.get(url, headers=headers, data=payload).json()

print(response)

In [ ]:
# Get all song ID inside the playlist
id_list = []

id_list = [song['track']['id'] for song in response['items']]

id_list

In [ ]:
# def getSongMetadata(song_id):
#     url = f"https://api.spotify.com/v1/tracks/{song_id}"

#     payload={}
#     headers = {
#         'Authorization': f'Bearer {AccessToken}'
#     }

#     songMetadata = requests.get(url, headers=headers, data=payload).json()

#     return songMetadata

In [ ]:
def getAudioFeature(song_id):
    url = f"https://api.spotify.com/v1/audio-features/{song_id}"

    payload = {}
    headers = {
        'Authorization': f'Bearer {AccessToken}'
    }

    songAudioFeature = requests.get(url, headers=headers, data=payload).json()

    return songAudioFeature

In [ ]:
# Version 2.0: List in Dictionary

songsInPlaylist = {
    'danceability': [],
    'energy': [],
    'key': [],
    'loudness': [],
    'mode': [],
    'speechiness': [],
    'acousticness': [],
    'instrumentalness': [],
    'liveness': [],
    'tempo': [],
    'valence': [],
}

def extractMetadata(AudioFeatureFunc):
    # Append all the informations to the list
    _ = AudioFeatureFunc
    songsInPlaylist['danceability'].append(_['danceability'])
    songsInPlaylist['energy'].append(_['energy'])
    songsInPlaylist['key'].append(_['key'])
    songsInPlaylist['loudness'].append(_['loudness'])
    songsInPlaylist['mode'].append(_['mode'])
    songsInPlaylist['speechiness'].append(_['speechiness'])
    songsInPlaylist['acousticness'].append(_['acousticness'])
    songsInPlaylist['instrumentalness'].append(_['instrumentalness'])
    songsInPlaylist['liveness'].append(_['liveness'])
    songsInPlaylist['tempo'].append(_['tempo'])
    songsInPlaylist['valence'].append(_['valence'])


In [ ]:
# # Version 1.1: Automatic, with List
# Variable definitions
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
tempo = []
valence = []

In [ ]:
# # Version 1.1: One iteration for all
# def extractMetadata(AudioFeatureFunc):
#     # Append all the informations to the list
#     _ = AudioFeatureFunc
#     danceability.append(_['danceability'])
#     energy.append(_['energy'])
#     key.append(_['key'])
#     loudness.append(_['loudness'])
#     mode.append(_['mode'])
#     speechiness.append(_['speechiness'])
#     acousticness.append(_['acousticness'])
#     instrumentalness.append(_['instrumentalness'])
#     liveness.append(_['liveness'])
#     tempo.append(_['tempo'])
#     valence.append(_['valence'])

In [ ]:
for id in id_list:
    extractMetadata(getAudioFeature(id))

## Standart Normalisation
Normalise the data before visualising it

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create an instance of StandardScaler
scaler = StandardScaler()

# Loop through the keys in the dictionary
for key in songsInPlaylist.keys():
    # Get the list of values for the current key
    values = songsInPlaylist[key]
    
    # Convert the list to a 2D array (required for StandardScaler)
    values = [[v] for v in values]
    
    # Fit and transform the values using StandardScaler
    normalized_values = scaler.fit_transform(values)
    
    # Update the dictionary with the normalized values
    songsInPlaylist[key] = [v[0] for v in normalized_values]

## Plotting into Scatter Plots

In [ ]:
def generate_random_hex_color():
    # Generate random integers for R, G, B values
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    # Convert RGB values to hexadecimal
    hex_color = '#%02x%02x%02x' % (r, g, b)
    return hex_color

In [ ]:
for _feature in songsInPlaylist:
    random_hex_color = generate_random_hex_color()
    for _values in songsInPlaylist[_feature]:
        plt.plot(_feature, _values, 'o', c=random_hex_color)
        plt.xticks(rotation=45, ha='right')

plt.show()

# Feature Guide
#### [0 to 1] danceability
0 is least danceable, 1 is most danceable

#### [0 to 1] energy
represent a perceptual measure of intensity and activity.

#### [-1 to 11] key
key the track is in

#### [-60 to 0db] loudness
the quality of a sound that is primary psychological correlate of physical strength (amplitude)

#### [0 or 1] mode
chord usage, 0 for minor, 1 for major

#### [0 to 1] speechiness
the higher means the more speech is in use in the track.

For example, audiobooks will have 1 in its speechiness. 

x > 0.66			Track is made from spoken words
0.33 > x > 0.66	Track contains both music and speech
x < 0.33			Represent music and other non-speech-like tracks

#### [0 to 1] acousticness 
measure the level of acoustic confidence in the track

#### instrumentalness
higher instrumentalness means no vocal content.

x > 5	Instrumental Tracks

#### [float] liveness
presence of audience in the recording/track. The higher the value means there is a higher probability that the track is taken from a live performance

#### [integer] tempo
BPM Counter

#### [0 to 1] valence
Measure the musical positiveness conveyed by the track.